# **Bibliotecas**

In [ ]:
import pandas as pd #usado para manipular os dados da tabela, retonados pelo gcp
from google.cloud import bigquery #usado para fazer consultas no gcp
from google.colab import auth #usado para autenticação
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.client import GoogleCredentials
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload
from io import BytesIO


# **Autenticação**

In [ ]:
auth.authenticate_user()
print('Authenticated')

# Criação das credenciais da API do Drive
creds = GoogleCredentials.get_application_default()
if creds.create_scoped_required():
    creds = creds.create_scoped(['https://www.googleapis.com/auth/drive'])

# Criação da instância do serviço do Drive
service = build('drive', 'v3', credentials=creds)

Authenticated


# **Carregar demandas de dados**

In [ ]:
client = bigquery.Client(project='cohesive-geode-254820')

query = """
INSERT INTO `cohesive-geode-254820.testes.querys_auto`

SELECT * FROM `cohesive-geode-254820.testes.querys_aut` WHERE id NOT IN (SELECT DISTINCT id FROM `cohesive-geode-254820.testes.querys_auto`)
;"""

query_job = client.query(query)

In [ ]:
client = bigquery.Client(project='cohesive-geode-254820')

# Execute uma consulta no BigQuery para obter as demandas solicitadas pelo formulário salvas no bq
query = """
SELECT *
FROM `cohesive-geode-254820.testes.querys_auto`
where data_sol is not null and status_sol = 'pendente'
"""
# Executar a consulta e recuperar os resultados.
query_job = client.query(query)
rows = query_job.result()

# Converter as linhas em uma lista de dicionários.
row_list = [dict(row) for row in rows]

# Converter a lista de dicionários em um objeto pandas.DataFrame.
df = pd.DataFrame(row_list)


# **LOOP PARA GERAR A STRING DA CONSULTA, PROCESSAR NO GCP E COMPARTILHAR COM O USUÁRIO**

In [ ]:
# Defines variáveis do LOOP
ind_i = int(df['id'].min()) # índice inicial
ind_f = int(df['id'].max()) # índice final
qtl = ind_f - ind_i # quantidade de consultas que serão realizadas
ls_ids = ','.join(map(lambda x: f"'{x}'", df['id'].tolist())) # string com os ids que serão processados
i = 0

while qtl >= i:
# Define as variáveis únicas da consulta a ser realizada
  data_inicial = df.loc[i, 'data_inicial']
  data_final = df.loc[i, 'data_final']
  filtro = df.loc[i, 'filtro']
  tipo_filtro = df.loc[i, 'tipo_filtro']
  campos = df.loc[i, 'dimensoes']
  caminho = df.loc[i, 'caminho']
  tp_data = ''
  if df.loc[i, 'tipo_data'] == 'APROVADA':
    tp_data = 'dthr_aprovacao'
  elif df.loc[i, 'tipo_data'] == 'COMPRA':
    tp_data = 'dthr_compra'


  # define consulta
  consulta = f'select {campos} \
  from {caminho} \
  where date({tp_data}) between "{data_inicial}" and "{data_final}" and {tipo_filtro} = "{filtro}"'

  # RODA A CONSULTA
  query = f'{consulta}'
  # Executar a consulta e recuperar os resultados.
  query_job = client.query(query)
  rows = query_job.result()

  # Converter as linhas em uma lista de dicionários.
  row_list = [dict(row) for row in rows]

  # Converter a lista de dicionários em um objeto pandas.DataFrame.
  dfe = pd.DataFrame(row_list)

  # Criação do arquivo CSV na memória
  csv_file = BytesIO()
  dfe.to_csv(csv_file, index=False)
  csv_file.seek(0)

  # Configurações do arquivo no Google Drive
  #file_metadata = {'name': f'dados{i}.csv', 'parents': ['10_zHbdf9W1mV_U5LreFNJZGOO5-0784d']}
  file_metadata = {'name': f'dados{i}.csv', 'parents': ['1jTN_Rubhgv1TY2-35HdUX_1h5dY69y2Q']}
  media = MediaIoBaseUpload(csv_file, mimetype='text/csv')

  # Upload do arquivo para o Drive
  file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

  # Adição de permissão de leitura no arquivo para um usuário específico
  file_id = file['id']
  user_email = df.loc[i,'email']
  permission = {
      'type': 'user',
      'role': 'writer',
      'emailAddress': user_email
  }
  service.permissions().create(fileId=file_id, body=permission).execute()

  i += 1 # FIM DO LOOP


ATUALIZA O STATUS DA DEMANDA

In [ ]:
# query para atualizar os status
query = f'update `cohesive-geode-254820.testes.querys_auto` \
set \
status_sol = "finalizado" \
where id in ({ls_ids})'

# Executar a consulta e recuperar os resultados.
query_job = client.query(query)